# Recommender System

This notebook demonstrates user-based and item-based collaborative filtering recommender system using cosine similarity.
Missing values are represented by `'?'`.


In [1]:
import numpy as np
import pandas as pd

# Editable ratings matrix with '?' for missing values

k =5 

ratings_input = {
    "Product A": [2, 3,'?', 1, '5'],
    "Product B": [3, 1, 5, 4, '?'],
    "Product C": ['?', 2, 3, 1, 5],
    "Product D": [4, 3, 2, '?', 4],
    "Product E": [5, '?', 3, 4, 1]
}

    
users = [f"User{i+1}" for i in range(5)]
ratings_df = pd.DataFrame(ratings_input, index=users)
ratings = ratings_df.replace('?', np.nan).astype(float)
ratings_display = ratings_df.copy()
ratings_display


C:\Users\Tirtham\AppData\Local\Temp\ipykernel_34280\502505628.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ratings = ratings_df.replace('?', np.nan).astype(float)


,Product A,Product B,Product C,Product D,Product E
User1,2,3,?,4,5
User2,3,1,2,3,?
User3,?,5,3,2,3
User4,1,4,1,?,4
User5,5,?,5,4,1


## Cosine Similarity

Cosine similarity is computed using the formula:

$$
\cos(\theta) = \frac{\vec{A} \cdot \vec{B}}{\|\vec{A}\| \|\vec{B}\|}
$$

This formula directly computes the cosine of the angle between two vectors without measuring the angle itself.
It is efficient and works well with sparse data by ignoring missing values.

##  Computational Benefits of Cosine Similarity

Efficient: Uses dot products and norms.

Sparse-friendly: Ignores missing values.

Pattern-focused: Captures rating behavior.

Scalable: Works well with large datasets.

In [15]:
def cosine_similarity_nan(vec1, vec2):
    mask = ~np.isnan(vec1) & ~np.isnan(vec2)
    if np.sum(mask) == 0:
        return 0
    v1, v2 = vec1[mask], vec2[mask]
    dot = np.dot(v1, v2)
    norm1, norm2 = np.linalg.norm(v1), np.linalg.norm(v2)
    return dot / (norm1 * norm2) if norm1 > 0 and norm2 > 0 else 0


## User Similarity Matrix
We compute pairwise cosine similarity between users based on their ratings.

In [21]:
user_sim = pd.DataFrame(np.ones((5,5)), index=users, columns=users)
for u1 in users:
    for u2 in users:
        if u1 != u2:
            user_sim.loc[u1, u2] = cosine_similarity_nan(ratings.loc[u1].values, ratings.loc[u2].values)
user_sim


,User1,User2,User3,User4,User5
User1,1.000000,0.894630,0.871780,0.960130,0.713068
User2,0.894630,1.000000,0.737043,0.566947,0.970998
User3,0.871780,0.737043,1.000000,0.929131,0.855337
User4,0.960130,0.566947,0.929131,1.000000,0.462069
User5,0.713068,0.970998,0.855337,0.462069,1.000000


In [24]:
items = ratings.columns
item_sim = pd.DataFrame(np.ones((5,5)), index=items, columns=items)
for i1 in items:
    for i2 in items:
        if i1 != i2:
            item_sim.loc[i1, i2] = cosine_similarity_nan(ratings[i1].values, ratings[i2].values)
item_sim

,Product A,Product B,Product C,Product D,Product E
Product A,1.000000,0.681385,0.987541,0.937385,0.535264
Product B,0.681385,1.000000,0.866025,0.784706,0.920000
Product C,0.987541,0.866025,1.000000,0.963960,0.596694
Product D,0.937385,0.784706,0.963960,1.000000,0.845154
Product E,0.535264,0.920000,0.596694,0.845154,1.000000


## User-Based Predictions

Predicted rating of User $u$ for Product $i$ is calculated as follows:

$$
\hat{r}_{ui} = \frac{\sum\limits_{v\neq u} \text{sim}(u, v) \cdot r_{vi}}{\sum\limits_{v \neq u} |\text{sim}(u, v)|}
$$

## Why Is Modulus Used in the Computation?
 
The modulus (absolute value) is used in the denominator of the prediction formula to ensure that negative similarities (if any) still contribute to the total weight.
 
It prevents division by zero.

It maintains numerical stability and proper scaling of predictions.


## Example (Explicit calculation):

In the input ratings, there is no rating by User 1 for Product C. This rating is calculated as a weighted average of the ratings of other users who have rated Product C, that is, User 2, User 3, User 4, User 6 and User 7.

$$
\hat{r}_{1C} = \frac{\sum\limits_{v\neq 1} \text{sim}(1, v) \cdot r_{vC}}{\sum\limits_{v \neq 1} |\text{sim}(1, v)|}
$$

$$
\hat{r}_{1C} = \frac{(0.964764*2)+(0.946864*5)+(0.684762*4)+(0.729397*3)+(0.546268*1)}{0.964764+0.946864+0.684762+0.729397+0.546268}
$$

$$
\hat{r}_{1C} = 3.13460
$$
<br><br>

The predictions for the other missing ratings are calculated in a similar way.

In [27]:
user_recs = {}
for user in users:
    scores = {}
    for item in items:
        if np.isnan(ratings.loc[user, item]):
            total_sim, weighted_sum = 0, 0
            for other in users:
                if other != user and not np.isnan(ratings.loc[other, item]):
                    sim = user_sim.loc[user, other]
                    weighted_sum += sim * ratings.loc[other, item]
                    total_sim += abs(sim)
            if total_sim > 0:
                scores[item] = (weighted_sum / total_sim).round(3)
    if scores:
        print(f"{user} predictions: {scores}")
        best_item = max(scores, key=scores.get)
        user_recs[user] = (best_item, scores[best_item])
pd.DataFrame(user_recs, index=["Recommended Item", "Predicted Rating"]).T


User1 predictions: {'Product C': 2.596}
User2 predictions: {'Product E': 3.131}
User3 predictions: {'Product A': 2.7}
User4 predictions: {'Product D': 3.169}
User5 predictions: {'Product B': 3.077}


,Recommended Item,Predicted Rating
User1,Product C,2.596
User2,Product E,3.131
User3,Product A,2.7
User4,Product D,3.169
User5,Product B,3.077


## Item-Based Predictions (Product-Based Predictions)

Predicted rating of User $u$ for Product $i$ is calculated as follows:

$$
\hat{r}_{ui} = \frac{\sum\limits_{j\neq i} \text{sim}(i, j) \cdot r_{uj}}{\sum\limits_{j\neq i} |\text{sim}(i, j)|}
$$




## Example (Explicit calculation):

In the input ratings, there is no rating by User 1 for Product C. This rating is calculated as a weighted average of the ratings of User 1 for other products he/she has rated, that is, Product A, Product B, Product D, Product E and Product G.

$$
\hat{r}_{1C} = \frac{\sum\limits_{j\neq C} \text{sim}(C, j) \cdot r_{1j}}{\sum\limits_{j \neq C} |\text{sim}(C, j)|}
$$

$$
\hat{r}_{1C} = \frac{(0.666667*5)+(0.925820*3)+(0.603860*1)+(0.960000*2)+(0.976187*4)}{0.666667+0.925820+0.603860+0.960000+0.976187}
$$

$$
\hat{r}_{1C} = 3.0343133
$$
<br><br>

The predictions for the other missing ratings are calculated in a similar way.

In [30]:
item_recs = {}
for user in users:
    scores = {}
    for item in items:
        if np.isnan(ratings.loc[user, item]):
            total_sim, weighted_sum = 0, 0
            for rated_item in items:
                if not np.isnan(ratings.loc[user, rated_item]) and item != rated_item:
                    sim = item_sim.loc[item, rated_item]
                    weighted_sum += sim * ratings.loc[user, rated_item]
                    total_sim += abs(sim)
            if total_sim > 0:
                scores[item] = weighted_sum / total_sim
    if scores:
        print(f"{user} predictions: {scores}")
        best_item = max(scores, key=scores.get)
        item_recs[user] = (best_item, scores[best_item])
pd.DataFrame(item_recs, index=["Recommended Item", "Predicted Rating"]).T


User1 predictions: {'Product C': 3.3426278420429374}
User2 predictions: {'Product E': 2.158923261510137}
User3 predictions: {'Product A': 3.1354050644554796}
User4 predictions: {'Product D': 2.3846774524059993}
User5 predictions: {'Product B': 3.627138380501174}


,Recommended Item,Predicted Rating
User1,Product C,3.342628
User2,Product E,2.158923
User3,Product A,3.135405
User4,Product D,2.384677
User5,Product B,3.627138


## EXERCISES

Take $R = \text{Digital root of your Roll Number}$ and $k = (R \mod 4) + 1$.

For this value of $k$, implement the Recommender System which includes 

1. one explicit calculation of cosine similarity between two users. Verify your answer.

2. one explicit calculation of cosine similarity between two products. Verify your answer.

3. one explicit calculation of user-based prediction. Verify your answer.

4. one explicit calculation of product-based prediction. Verify your answer.

## NOTE:

Cosine similarity is also widely used in Natural Language Processing (NLP) to measure how similar two text (word) vectors are, independent of their magnitude. It helps capture how close two vectors are in high-dimensional space, reflecting the similarity of meanings between words, sentences, or documents.

### Roll No: 13
### Digital Root (R) = 1+3= 4
### k = (4 mod 4) + 1= 1
